## Grids

Es gibt viele Problemstellungen, bei denen wir durch eine 2D-Spielfeld (grid) navigieren müssen.

Beispiel: Wir wollen den kürzesten Pfad vom Start zum Ziel finden.

    #:  Wand
    S:  Start
    G:  Ziel (goal)


In [103]:
%%writefile maze1.txt
##########
#   #    #
#   ##   #
# #Z#  ###
# ## #   #
#   S #  #
#     #  #
#        #
##########

Overwriting maze1.txt


#### Einlesen des Grids

Wenn wir das grid nicht verändern wollen, können wir die grid-Daten als eine Liste von Strings einlesen.

In [105]:
f = open('maze3.txt')       
grid = f.read().splitlines()
f.close()
grid

['###################################################################',
 '#                                                                 #',
 '#                                                                 #',
 '#                                                                 #',
 '#                                                                 #',
 '#                                                                 #',
 '#                                                                 #',
 '#                                                                 #',
 '#                                                                 #',
 '#                                                                 #',
 '#                                                                 #',
 '#                                                                 #',
 '#S                                                               Z#',
 '#                                                             

Wir ermitteln die Höhe und Breite des Grids.

In [106]:
height = len(grid)
width = len(grid[0])
print(f'Höhe = {height}, Breite = {width}')

Höhe = 26, Breite = 67


#### Koordinaten im Grid

Der Startpunkt ist in der Zeile mit Index 5, Spalte mit Index 4

In [32]:
grid[5][4]             # Zeile mit Index 5, Spalte mit Index 4

'S'

Die Koordinaten können je nach Problemstellung unterschiedliche Positionen bedeuten, je nachdem wo der Nullpunkt liegt und in welche Richtung die Achsen zeigen. 

Häufig ist folgendes Setting: Der Nullpunkt liegt oben links und wird mit (0,0) bezeichnet. Die x-Achse zeigt nach rechts, die y-Achse nach unten. Damit hat der Punkt S die Koordinaten (4,5). 
Für dieses Setting gilt dann: An der Koordinate (x,y) steht der Wert grid[y][x].

Zur Vereinfachung der folgenden Überlegungen vereinbaren wir:
Mit *Zeile x* bezeichnen wir die Zeile mit Index x. Mit *Spalte y* bezeichnen wir die Spalte mit Index y. Die Koordinate (x,y) oder Position (x,y) bezeichnet die Stelle in Zeile x und Spalte y.
Damit zeigt also die x-Achse nach unten und die y-Achse nach rechts. An der Koordinate (x,y) steht der Wert grid[x][y].


#### Start und Ziel ermitteln    

In [107]:
for x in range(height):
    for y in range(width):
        if grid[x][y] == 'S':
            start = (x,y)
        elif grid[x][y] == 'Z':
            ziel = (x,y)

print(f'{start=}, {ziel=}')

start=(12, 1), ziel=(12, 65)


#### Nachbarschaft

In unseren Algorithmen benötigen wir immer wieder die Nachbarn zu einer bestimmten Position (=Koordinate).
Wir schreiben uns dazu eine Funktion *nb*. Wir gehen im folgenden davon aus, dass das grid sich nicht verändert. Deshalb nutzen wir (zur Vereinfachung) *grid*, *height* und *width* als globale Variablen. Außerdem spendieren wir uns noch eine globale Variable *dirs* mit den möglichen Bewegungsrichtungen


In [34]:
dirs = [(0,-1),(0,1),(-1,0),(1,0)]     # NSWE = hoch,runter,links,rechts

def nb(pos):
    '''
    pos: Tuple (x,y) - die Position im grid
    returns: Liste mit den möglichen Nachbarpositionen
    '''
    x, y = pos
    tmp = []
    for xd, yd in dirs:
        x1 = x + xd
        y1 = y + yd
        if 0 <= x1 < height and 0 <= y1 < width and grid[x1][y1] != '#':
            tmp.append((x1,y1))
    return tmp    

In [35]:
# Beispiele
print(nb((4,1)))
print(nb((5,1)))

[(3, 1), (5, 1)]
[(5, 2), (4, 1), (6, 1)]


### BFS Breitensuche

Die Breitensuche findet einen kürzesten Weg vom startstate zu einem goalstate. Das allgemeine Muster für die Breitensuche:

In [36]:
from collections import deque
def bfs(startstate):
    ''' 
    returns: Tuple (prev, state) 
        prev: dictionary mit den Vorgängern der untersuchten Spielstellungen,            
        state: Spielstellung, die den goaltest besteht
        wenn Ziel nicht gefunden: None, None
    '''   
    frontier =  deque([startstate])
    prev = {startstate:None}
    while frontier:
        state = frontier.popleft()  
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, None

def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]
    path.reverse()
    return path

def nextstates(state):
    '''
    state: Spielstellung
    returns:  Liste mit möglichen Folgestellungen zu state
    '''

def goaltest(state):
    '''
    state: Spielstellung
    returns: True, wenn state eine Lösung ist
    '''
    
#Aufruf:
#start = ...
#prev, goal = bfs(start)
#path = reconstructPath(prev,goal) 

#### Breitensuche für das Grid

Im konkreten Fall sehen die zu implementieren Funktionen wie folgt aus:

In [37]:
def nextstates(state):
    return nb(state)

def goaltest(state):
    return state == ziel


prev, goal = bfs(start)
path = reconstructPath(prev,goal)
print(path)

[(5, 4), (5, 3), (5, 2), (5, 1), (4, 1), (3, 1), (2, 1), (2, 2), (2, 3), (3, 3)]


Häufig möchte man die Folge von Aktionen nennen, die diesem Weg entspricht. Dazu implementieren wir noch die Funktion *getMove*

In [38]:
def getMove(s1, s2):
    '''
    returns: die Beschreibung des Übergangs von state s1 zu state s2
    '''
    x1,y1 = s1
    x2,y2 = s2
    if x1 < x2: return 'D'   # down
    if x1 > x2: return 'U'   # up
    if y1 < y2: return 'R'   # right
    if y1 > y2: return 'L'   # left

def getMoves(path):
    '''
    returns: Beschreibung des Pfads als eine Folge von Aktionen (Moves)
    '''
    moves = ''
    for i in range(len(path)-1):
        moves+=getMove(path[i],path[i+1])
    return moves

In [39]:
prev, goal = bfs(start)
path = reconstructPath(prev,goal)
print(getMoves(path))

LLLUUURRD


Wir lassen uns den gefunden Weg im Grid anzeigen. Die untersuchten Koordinaten in prev werden mit '.' bezeichnet.

In [77]:
def showPath(grid, path):
    grid0 = [list(s) for s in grid]
    for x,y in prev:
        grid0[x][y]='.'
    for x,y in path:
        grid0[x][y] = 'o'
    x,y = path[0]
    grid0[x][y] = 'S'
    x,y = path[-1]
    grid0[x][y] = 'Z'
    for row in grid0:
        print(''.join(row))
 
    print(f'prev = {len(prev)}, weglänge = {len(path)-1}')
    print("gefundener Weg: 'o', prev: '.'")

showPath(grid, path)

##########
#...#    #
#ooo##   #
#o#Z#  ###
#o##.#...#
#oooS.#..#
#.....#..#
#........#
##########
prev = 35, weglänge = 9
gefundener Weg: 'o', prev: '.'


#### Breitensuche fürs Grid - kompakte Version

In [71]:
from collections import deque
def bfsPath(grid,startChar='S',goalChars='Z',wallChar='#'):
    def bfs(startstate):
        frontier =  deque([startstate])
        prev = {startstate:None}
        while frontier:
            x, y = frontier.popleft()  
            if grid[x][y] in goalChars:
                return prev, (x,y)
            for xd, yd in dirs:
                x1 = x + xd
                y1 = y + yd
                if 0 <= x1 < height and 0 <= y1 < width and grid[x1][y1] != wallChar:
                    if (x1,y1) not in prev:
                        frontier.append((x1,y1))
                        prev[(x1,y1)] = (x,y)
        return None, None
    
    def reconstructPath(prev,goalstate):
        state = goalstate
        path = []
        while state is not None:
            path.append(state)
            state = prev[state]
        path.reverse()
        return path

    dirs = [(0,-1),(0,1),(-1,0),(1,0)]
    height = len(grid)
    width = len(grid[0])
    for x in range(height):
        for y in range(width):
            if grid[x][y] == startChar:
                start = (x,y)
 
    prev, goal = bfs(start)
    if not prev: return []                    # kein Pfad zum Ziel gefunden
    return reconstructPath(prev,goal)

In [95]:
f = open('maze2.txt')       
grid = f.read().splitlines()
f.close()
path = bfsPath(grid)
print(getMoves(path))

LLLLLLLLLLLLLLLLLLLLLDDDRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRUUUUUUU


Die folgende Funktion dient nur zu Verdeutlichung des Ablaufs

In [97]:
from collections import deque
def bfsPathShow(grid,startChar='S',goalChars='Z',wallChar='#'):
    def bfs(startstate):
        frontier =  deque([startstate])
        prev = {startstate:None}
        while frontier:
            x, y = frontier.popleft()  
            if grid[x][y] in goalChars:
                return prev, (x,y), frontier
            for xd, yd in dirs:
                x1 = x + xd
                y1 = y + yd
                if 0 <= x1 < height and 0 <= y1 < width and grid[x1][y1] != wallChar:
                    if (x1,y1) not in prev:
                        frontier.append((x1,y1))
                        prev[(x1,y1)] = (x,y)
        return None, None, None
    
    def reconstructPath(prev,goalstate):
        state = goalstate
        path = []
        while state is not None:
            path.append(state)
            state = prev[state]
        path.reverse()
        return path

    dirs = [(0,-1),(0,1),(-1,0),(1,0)]
    height = len(grid)
    width = len(grid[0])
    for x in range(height):
        for y in range(width):
            if grid[x][y] == startChar:
                start = (x,y)
 
    prev, goal, frontier = bfs(start)
    if not prev: return                # kein Pfad zum Ziel gefunden
    path = reconstructPath(prev,goal)

    grid0 = [list(s) for s in grid]
    for x,y in prev:
        grid0[x][y]='.'
    for x,y in frontier:
        grid0[x][y]='~'
    for x,y in path:
        grid0[x][y] = 'o'
    x,y = path[0]
    grid0[x][y] = 'S'
    x,y = path[-1]
    grid0[x][y] = 'Z'
    for row in grid0:
        print(''.join(row))
 
    print(f'weglänge = {len(path)-1}, prev = {len(prev)}, frontier = {len(frontier)}')
    print("gefundener Weg: 'o', prev: '.', frontier: '~'")

In [90]:
bfsPathShow(grid)

###################################################################
#...................................~                             #
#....................................~                            #
#.....................................~                           #
#......................................~#                         #
#.......................................#                         #
#.......................................#                         #
#.......................................#                         #
#.......................................#############             #
#.......................................#                         #
#.......................................#                         #
#.......................................#                         #
#.......................................#                         #
#.......................................#~                        #
#.......................................#.~     

In [146]:
f = open('maze3.txt')       
grid = f.read().splitlines()
f.close()
path = bfsPath(grid)
print(getMoves(path))
print()
bfsPathShow(grid)

RRRRRRRRRRRRRRRRRRRRRRRRRRUURRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRDD

###################################################################
#.........................................................~       #
#..........................................................~      #
#...........................................................~     #
#............................................................~    #
#.............................................................~   #
#..............................................................~  #
#...............................................................~~#
#.................................................................#
#.................................................................#
#..........................ooooooooooooooooooooooooooooooooooooooo#
#..........................o#....................................o#
#Soooooooooooooooooooooooooo#....................................Z#
#...........................#.................

### AStar

Der A*-Algorithmus gehört zu den *informed search*-Algorithmen. Das bedeutet, wir haben eine Heuristik zur Verfügung, die uns angibt, wie weit wir vermutlich vom Ziel entfernt sind. Wenn die Heuristik die tatsächlichen Kosten für den noch zurückzulegenden Weg nicht überschätzt, findet A* einen optimalen Weg.

Einem state wird ein A*-value zugeordnet, das ist die Summe aus den Kosten für den schon zurückgelegten Weg und den noch vermuteten Kosten laut Heuristik. Da wir immer den state mit dem niedrigsten A*-value aus der frontier holen wollen, ist die frontier als Heap organisiert.

In dem Beispiel nehmen wir an, das (wie im grid) jeder Schritt die Kosten 1 hat.
Als Heuristik nehmen wir die Manhattendistanz zwischen einer Position und dem Ziel.

In [116]:
from heapq import heappop, heappush
def astar(s):
    frontier =[(h(s),s)]  
    prev = {s:None}
    g = {s:0}                         # die Rückwärtskosten
    while frontier:
        _ ,state = heappop(frontier)  # die Kosten braucht man an der Stelle nicht
        if goaltest(state):
            return prev, state
        for v in nextstates(state):
            gg = g[state] + 1         # statt 1 könnten hier auch die Kosten von state nach v stehen
            if v not in prev or gg < g[v]:     # vielleicht waren wir schon bei v, aber mit höheren Kosten
                g[v] = gg                      # die (evtl neuen) Rückwärtskosten für v
                heappush(frontier,(g[v]+h(v),v))  
                prev[v] = state
    return None, None

def nextstates(state):
    return nb(state)

def goaltest(state):
    return state == ziel

def h(state):
    '''
    state: Spielstellung
    returns: Vorwärtskosten laut Heuristik
    '''
    x1, y1 = state
    x2, y2 = ziel
    return abs(x1-x2) + abs(y1-y2)

#### AStar für Grids - kompakte Version

In [118]:
f = open('maze3.txt')       
grid = f.read().splitlines()
f.close()
for x in range(height):
    for y in range(width):
        if grid[x][y] == 'S':
            start = (x,y)
        elif grid[x][y] == 'Z':
            ziel = (x,y)

print(f'{start=}, {ziel=}')

prev, goal = astar(start)
print(getMoves(path))

start=(12, 1), ziel=(12, 65)
RRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR


In [147]:
from heapq import heappop, heappush
def astarPathShow(grid,startChar='S',goalChar='Z',wallChar='#'):
    def astar(s):
        frontier =[(h(s),s)]  
        prev = {s:None}
        g = {s:0}                         # die Rückwärtskosten
        while frontier:
            _ ,state = heappop(frontier)  # die Kosten braucht man an der Stelle nicht
            if goaltest(state):
                return prev, state, frontier
            for v in nb(state):
                gg = g[state] + 1         # statt 1 könnten hier auch die Kosten von state nach v stehen
                if v not in prev or gg < g[v]:     # vielleicht waren wir schon bei v, aber mit höheren Kosten
                    g[v] = gg                      # die (evtl neuen) Rückwärtskosten für v
                    heappush(frontier,(g[v]+h(v),v))  
                    prev[v] = state
        return None, None, None

    def h(state):
        x1, y1 = state
        x2, y2 = ziel
        return abs(x1-x2) + abs(y1-y2)
    
    def reconstructPath(prev,goalstate):
        state = goalstate
        path = []
        while state is not None:
            path.append(state)
            state = prev[state]
        path.reverse()
        return path

    dirs = [(0,-1),(0,1),(-1,0),(1,0)]
    height = len(grid)
    width = len(grid[0])
    for x in range(height):
        for y in range(width):
            if grid[x][y] == startChar:
                start = (x,y)
            if grid[x][y] == goalChar:
                ziel = (x,y)
 
    prev, goal, frontier = astar(start)
    if not prev: return                # kein Pfad zum Ziel gefunden
    path = reconstructPath(prev,goal)
 

    grid0 = [list(s) for s in grid]
    for x,y in prev:
        grid0[x][y]='.'
    for _,(x,y) in frontier:
        grid0[x][y]='~'
    for x,y in path:
        grid0[x][y] = 'o'
    x,y = path[0]
    grid0[x][y] = 'S'
    x,y = path[-1]
    grid0[x][y] = 'Z'
    for row in grid0:
        print(''.join(row))
 
    print(f'weglänge = {len(path)-1}, prev = {len(prev)}, frontier = {len(frontier)}')
    print("gefundener Weg: 'o', prev: '.', frontier: '~'")

In [148]:
f = open('maze3.txt')       
grid = f.read().splitlines()
grid = [s.strip() for s in grid]
f.close()
astarPathShow(grid)

 

###################################################################
#                                                                 #
#                                                                 #
#                                                                 #
#                                                                 #
#                                                                 #
#                                                                 #
#                                                                 #
#                                                                 #
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
#..........................ooooooooooooooooooooooooooooooooooooooo#
#..........................o#....................................o#
#Soooooooooooooooooooooooooo#....................................Z#
#...........................#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
#~~~~~~~~~~~~~~~~~~~~~~~~~~~                    